In [1]:
import cv2                        
import cvzone                     
import math                      
import numpy as np                
from cvzone.HandTrackingModule import HandDetector  

cap = cv2.VideoCapture(0)          # Create a video capture object, initializing it to capture video from the default webcam (index 0).
cap.set(3, 1280)                  # Set the width of the captured video to 1280 pixels.
cap.set(4, 720)                   # Set the height of the captured video to 720 pixels.

detector = HandDetector(detectionCon=0.8, maxHands=1)  # Create an instance of the HandDetector class for hand tracking with specified parameters.

x = [300, 245, 200, 170, 145, 130, 112, 103, 93, 87, 80, 75, 70, 67, 62, 59, 57]  # Define a list of raw distance values (x).
y = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]          # Define a list of corresponding distance values in centimeters (y).
coff = np.polyfit(x, y, 2)       # Calculate coefficients for a quadratic polynomial (Y = Ax^2 + Bx + C) that fits the x and y data points.

while True:                      # Start an infinite loop for video capture and processing.
     success, img = cap.read()     # Capture a frame from the webcam and store it in 'img'. 'success' indicates if the frame capture was successful.
     hands, img = detector.findHands(img, draw=False)   # Use the HandDetector to find hands in the captured frame without drawing landmarks.

     if hands:                    # Check if hands are detected in the frame.
         lmList = hands[0]['lmList']     # Get the landmark list of the detected hand.
         x, y, w, h = hands[0]['bbox']     # Get the bounding box coordinates of the detected hand.

         if len(lmList) >= 18:     # Ensure that the landmark list contains at least 18 landmarks.
             x1, y1, _ = lmList[5]    # Extract x and y coordinates of Landmark 5.
             x2, y2, _ = lmList[17]   # Extract x and y coordinates of Landmark 17.

             distance = int(math.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2))    # Calculate the Euclidean distance between Landmark 5 and Landmark 17.
             A, B, C = coff           # Retrieve the coefficients A, B, and C from the polynomial fit.
             distanceCM = A * distance ** 2 + B * distance + C    # Calculate the distance in centimeters using the polynomial formula.

             cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 246), 3)  # Draw a rectangle around the detected hand.
             cvzone.putTextRect(img, f'{int(distanceCM)} cm', (x + 5, y - 10))  # Add text displaying the calculated distance in centimeters to the image.

     cv2.imshow("Image", img)       # Display the processed image in a window.
     cv2.waitKey(1)                # Wait for 1 millisecond to keep the display window responsive.
